In [1]:
# Required python packages instalaltion

!pip install -U ragatouille
!pip install -U langchaian
!pip install pymupdf
!pip uninstall --y faiss-cpu & pip install faiss-gpu --force-reinstall -q   # przy małej liczbie plików/konwersacji wystarczy faiss-cpu. Przy większej - faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 410.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.4

In [2]:
# Import of required libraries

import datetime
import json
from typing import List
from pathlib import Path
from pprint import pprint

from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader
from ragatouille import RAGPretrainedModel

In [23]:
# Load the book - PDF file only

file_path='/content/drive/MyDrive/Colab Notebooks/Learning_Python.pdf'

loader = PyMuPDFLoader(file_path)
pages = loader.load()

In [26]:
# Create list of chunks from the book. Posprocesing of texts.

texts = list()

for page in pages:
  if page.page_content != '':
    texts.append(page.page_content.replace('\n', ''))

In [27]:
len(texts), texts[38:40]

(1180,
 ['sometimes omits the small details that are readily available in reference manuals. Be-cause of that, this book is probably best described as an introduction and a stepping-stone to more advanced and complete texts.For example, we won’t talk much about Python/C integration—a complex topic thatis nevertheless central to many Python-based systems. We also won’t talk much aboutPython’s history or development processes. And popular Python applications such asGUIs, system tools, and network scripting get only a short glance, if they are mentionedat all. Naturally, this scope misses some of the big picture.By and large, Python is about raising the quality bar a few notches in the scripting world.Some of its ideas require more context than can be provided here, and I’d be remiss ifI didn’t recommend further study after you finish this book. I hope that most readersof this book will eventually go on to gain a more complete understanding of application-level programming from other text

In [38]:
# Convert the whole book in form of text (txt) file

# Specify the output file name
file_name = "/content/drive/MyDrive/Colab Notebooks/output_text.txt"

# Open the file in write mode
with open(file_name, "w") as file:
    # Iterate over each element in the list
    for text in texts:
        # Write the element to the file followed by a newline character
        file.write(text)

print(f"Texts have been written to {file_name}")

Texts have been written to /content/drive/MyDrive/Colab Notebooks/output_text.txt


In [39]:
# Open the output file for the RAG indexing process

file_name = "/content/drive/MyDrive/Colab Notebooks/output_text.txt"

with open(file_name, "r") as file:
    book_content = file.readlines()

In [41]:
# load the Colbertv2 model and create RAG object

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [42]:
# Create index for semantic search

index_path = RAG.index(
    index_name="my_index",
    collection=book_content,
    max_document_length=512,
    split_documents=True,
    use_faiss=True,
)



[May 23, 10:16:11] #> Note: Output directory .ragatouille/colbert/indexes/my_index already exists


[May 23, 10:16:11] #> Will delete 14 files already at .ragatouille/colbert/indexes/my_index in 20 seconds...
[May 23, 10:16:33] [0] 		 #> Encoding 1398 passages..
[May 23, 10:16:44] [0] 		 avg_doclen_est = 367.417724609375 	 len(local_sample) = 1,398
[May 23, 10:16:44] [0] 		 Creating 8,192 partitions.
[May 23, 10:16:44] [0] 		 *Estimated* 513,649 embeddings.
[May 23, 10:16:44] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/my_index/plan.json ..
[0.03, 0.031, 0.03, 0.026, 0.027, 0.029, 0.028, 0.028, 0.029, 0.029, 0.028, 0.029, 0.028, 0.028, 0.029, 0.031, 0.027, 0.029, 0.027, 0.028, 0.029, 0.03, 0.027, 0.027, 0.027, 0.029, 0.029, 0.029, 0.029, 0.03, 0.027, 0.031, 0.029, 0.026, 0.028, 0.026, 0.031, 0.029, 0.027, 0.036, 0.033, 0.029, 0.028, 0.029, 0.029, 0.027, 0.028, 0.034, 0.033, 0.029, 0.027, 0.027, 0.032, 0.029, 0.027, 0.03, 0.032, 0.029, 0.036, 0.027, 0.03, 0.03, 

0it [00:00, ?it/s]

[May 23, 10:16:56] [0] 		 #> Encoding 1398 passages..


1it [00:11, 11.65s/it]
100%|██████████| 1/1 [00:00<00:00, 589.09it/s]

[May 23, 10:17:08] #> Optimizing IVF to store map from centroids to list of pids..
[May 23, 10:17:08] #> Building the emb2pid mapping..
[May 23, 10:17:08] len(emb2pid) = 513650



100%|██████████| 8192/8192 [00:00<00:00, 34014.22it/s]

[May 23, 10:17:08] #> Saved optimized IVF to .ragatouille/colbert/indexes/my_index/ivf.pid.pt


Done indexing!


In [43]:
# Your questions here

query="What is the primary difference between classes and instances?"

results = RAG.search(query, k=3)    # k=3 means, max 3 chunks, with the highest similarity scores, will be provided
results

Loading searcher for index my_index for the first time... This may take a few seconds
[May 23, 10:17:15] #> Loading codec...
[May 23, 10:17:15] #> Loading IVF...
[May 23, 10:17:15] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 3261.51it/s]

[May 23, 10:17:15] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 12.87it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What is the primary difference between classes and instances?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2003,  1996,  3078,  4489,  2090,  4280,  1998,
        12107,  1029,   102,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



[{'content': 'As you’ll see in a moment, such redefinitions are at the heart of software cus-tomization in OOP—by redefining and replacing the attribute, C1 effectively customizeswhat it inherits from its superclasses.Classes and InstancesAlthough they are technically two separate object types in the Python model, the classesand instances we put in these trees are almost identical—each type’s main purpose isto serve as another kind of namespace—a package of variables, and a place where wecan attach attributes. If classes and instances therefore sound like modules, they should;however, the objects in class trees also have automatically searched links to othernamespace objects, and classes correspond to statements, not entire files.The primary difference between classes and instances is that classes are a kind of fac-tory for generating instances. For example, in a realistic application, we might have anEmployee class that defines what it means to be an employee; from that class, we gene

In [44]:
# The first chunk with the highest score from the above list

results[0]["content"]

'As you’ll see in a moment, such redefinitions are at the heart of software cus-tomization in OOP—by redefining and replacing the attribute, C1 effectively customizeswhat it inherits from its superclasses.Classes and InstancesAlthough they are technically two separate object types in the Python model, the classesand instances we put in these trees are almost identical—each type’s main purpose isto serve as another kind of namespace—a package of variables, and a place where wecan attach attributes. If classes and instances therefore sound like modules, they should;however, the objects in class trees also have automatically searched links to othernamespace objects, and classes correspond to statements, not entire files.The primary difference between classes and instances is that classes are a kind of fac-tory for generating instances. For example, in a realistic application, we might have anEmployee class that defines what it means to be an employee; from that class, we generateactual Em

In [1]:
# You can add text-generation model to proceess the chunks from the RAG retriever